# **TelecomX_parte2_Latam**

## Objetivo

---



Tu nueva misión es desarrollar modelos predictivos capaces de prever qué clientes tienen mayor probabilidad de cancelar sus servicios.

La empresa quiere anticiparse al problema de la cancelación, y te corresponde a ti construir un pipeline robusto para esta etapa inicial de modelado.

\

## Metas del proyecto

---


- Preparar los datos para el modelado (tratamiento, codificación, normalización).

- Realizar análisis de correlación y selección de variables.

- Entrenar dos o más modelos de clasificación.

- Evaluar el rendimiento de los modelos con métricas.

- Interpretar los resultados, incluyendo la importancia de las variables.

- Crear una conclusión estratégica señalando los principales factores que influyen en la cancelación.


\

## Habilidades a desarrollar

---

- Preprocesamiento de datos para Machine Learning
- Construcción y evaluación de modelos predictivos
- Interpretación de resultados y entrega de insights
- Comunicación técnica con enfoque estratégico



# 1.Preparación de los datos

## Extracción del Archivo Tratado

In [ ]:
import pandas as pd
datos =  pd.read_csv('/content/datos_tratados.csv')

In [ ]:
datos.shape

In [ ]:
datos.sample(2)

In [ ]:
datos.info()

##Eliminación de Columnas Irrelevantes

In [ ]:
datos = datos.drop(columns='clientesID', axis = 1)
datos.sample(2)

##Enconding

In [ ]:
datos['genero'] = datos['genero'].replace({'Hombre':0,'Mujer':1})
datos.head(1)

In [ ]:
categoricas = ['proveedor_internet','contrato','forma_pago']
datos_cod = pd.get_dummies(data = datos, columns = categoricas, dtype = int)
datos_cod.head(1)

## Verificación de la Proporción de Cancelación (Churn)


Calcula la proporción de clientes que cancelaron en relación con los que permanecieron activos. Evalúa si existe un desbalance entre las clases, ya que esto puede impactar en los modelos predictivos y en el análisis de los resultados.

In [ ]:
datos['estadia_empresa'].value_counts()

In [ ]:
datos['estadia_empresa'].value_counts(normalize = True)

##Balanceo de Clases

In [ ]:
datos_cod['gastos_totales'] = datos_cod['gastos_totales'].fillna(datos_cod['gastos_totales'].median())

X = datos_cod.drop(columns='estadia_empresa', axis = 1)
y = datos_cod['estadia_empresa']

In [ ]:
from imblearn.over_sampling import SMOTE

oversampling = SMOTE()
X_balanceada, y_balanceada = oversampling.fit_resample(X, y)

y_balanceada.value_counts(normalize = True)

## Normalización o Estandarización (si es necesario)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

normalizacion = MinMaxScaler()
#X_train_normalizada = normalizacion.fit_transform(X_train)

# 2.Correlación y Selección de Variables

## Análisis de Correlación

In [ ]:
correlacion = datos_cod.corr()
correlacion_estadia = correlacion['estadia_empresa'].sort_values(ascending = False)
display(correlacion_estadia)

## Análisis Dirigido

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Tiempo de contrato × Cancelación
sns.boxplot(x='estadia_empresa', y='meses_contrato', data=datos_cod)
plt.title('Tiempo de contrato según cancelación')
plt.xlabel('Estadia en la empresa (1 = Se quedó, 0 = Canceló)')
plt.ylabel('Meses de contrato')
plt.show()

Al observar el boxplot, se aprecia que los clientes que cancelaron el servicio (estadia_empresa = 0) tienden a tener más meses de contrato, mientras que los que se quedaron (estadia_empresa = 1) tienen contratos menos largos. Esto sugiere que una mayor permanencia está asociada con msyor probabilidad de cancelación.

In [ ]:
# Gasto total × Cancelación
sns.boxplot(x='estadia_empresa', y='gastos_totales', data=datos_cod)
plt.title('Gasto total según cancelación')
plt.xlabel('Estadia en la empresa (1 = Se quedó, 0 = Canceló)')
plt.ylabel('Gastos totales')
plt.show()

En el gráfico de gasto total, se puede observar que quienes gastan más suelen permanecer menos tiempo con la empresa. Esto puede indicar que clientes de alto valor no están comprometidos o satisfechos con el servicio.

# 3.Modelado Predictivo

## Separación de Datos

In [ ]:
datos.shape

In [ ]:
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split #permite separar los datos

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=21)

baseline = DummyRegressor(strategy='mean')
baseline.fit(X_train, y_train)

##Creación de Modelos

Crea al menos dos modelos diferentes para predecir la cancelación de clientes.

- Un modelo puede requerir normalización, como Regresión Logística o KNN.

- El otro modelo puede no requerir normalización, como Árbol de Decisión o Random Forest.


 La decisión de aplicar o no la normalización depende de los modelos seleccionados. Ambos modelos pueden ser creados sin normalización, pero también es una opción combinar modelos con y sin normalización.


**Justificación:**
- Regresión Logística / KNN: Estos modelos son sensibles a la escala de los datos, por lo que la normalización es importante para que los coeficientes o las distancias se calculen correctamente.

- Árbol de Decisión / Random Forest: Estos modelos no dependen de la escala de los datos, por lo que no es necesario aplicar normalización.

Si decides normalizar los datos, deberías explicar cómo esta etapa asegura que los modelos basados en distancia o en optimización de parámetros no se vean sesgados por la magnitud de las variables.

### Normalización

In [ ]:
X_train_normalizada = normalizacion.fit_transform(X_train)

### Modelo bosques aleatorios

In [ ]:
from sklearn.ensemble import RandomForestClassifier

modelo_bosques = RandomForestClassifier(max_depth = 5, random_state=21)
modelo_bosques.fit(X_train, y_train)

modelo_bosques.score(X_test, y_test)

### Modelo árbol de decisión

In [ ]:
from sklearn.tree import DecisionTreeClassifier

X, X_test2, y, y_test2 = train_test_split(X, y, test_size=0.3, stratify=y, random_state=21)
X_train2, X_val, y_train2, y_val = train_test_split(X, y, stratify=y, random_state=21)

modelo_arbol = DecisionTreeClassifier(max_depth=10)
modelo_arbol.fit(X_train, y_train)
modelo_arbol.score(X_val, y_val)

## Evaluación de lo Modelos

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
import sklearn.metrics as ConfusionMatrixDisplay

In [ ]:
from sklearn.metrics import mean_absolute_error # Error absoluto medio
from sklearn.metrics import mean_squared_error # Raiz cuadrada del error
from sklearn.metrics import r2_score # Coeficiente de determinación

y_pred_dummy = baseline.predict(X_test)

def calcular_regresion(y_test, y_pred):
  rmse = mean_squared_error(y_test, y_pred)
  mae = mean_absolute_error(y_test, y_pred)
  r2 = r2_score(y_test, y_pred)
  metricas = {'RMSE':round(rmse**(1/2), 4),
              'MAE':round(mae, 4),
              'R2':round(r2, 4)}

  return metricas

In [ ]:
def calcular_metricas (y_test, y_pred):
  exactitud = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred)
  matriz = confusion_matrix(y_test, y_pred)

  print(f'Exactitud: {exactitud:.2f}')
  print(f'Precisión: {precision:.2f}')
  print(f'Recall: {recall:.2f}')
  print(f'F1-score: {f1:.2f}')
  print('\nMatriz de confusión:')
  print(matriz)

###Modelo bosques aleatorios

In [ ]:
y_pred = modelo_bosques.predict(X_test)

In [ ]:
calcular_metricas (y_test, y_pred)


In [ ]:
matriz = confusion_matrix(y_test, y_pred)
visulizacion = ConfusionMatrixDisplay.ConfusionMatrixDisplay(matriz)
visulizacion.plot();

###Modelo árbol

In [ ]:
y_val_pred = modelo_arbol.predict(X_val)

In [ ]:
calcular_metricas (y_val, y_val_pred)

In [ ]:
matriz = confusion_matrix(y_val, y_val_pred)
visulizacion = ConfusionMatrixDisplay.ConfusionMatrixDisplay(matriz)
visulizacion.plot();

###COMPARACIÓN

- **Accuracy (exactitud):**
 - Árbol: clasifica correctamente el 74% de los casos.
 - Bosques: clasifica correctamente el 66%.


- **Precisión:**
 - Árbol: de todas las predicciones positivas (clientes que no cancelan), el 49% fueron correctas.
 - Bosques: solo el 24%, es decir, predice muchos falsos positivos.

- **Recall:**
 - Árbol: identifica correctamente al 50% de los que se quedaron.
 - Bosques: apenas reconoce al 16%.

- **F1-Score:**
 - Árbol: buen equilibrio entre precisión y recall (0.5).
 - Bosques: mal equilibrio (0.19).


 - **Matriz**
  - Árbol:
    - Bastante balanceado, con un número aceptable de verdaderos positivos.
    - Buen equilibrio entre errores de tipo I (FP) y tipo II (FN).
  - Bosques:
    - Muchísimos falsos negativos (474), es decir, predice mal a los clientes que se quedaron.
    - También tiene bastantes falsos positivos.
    - Pésima sensibilidad (recall) — casi no detecta a los que se quedan.



#### Overfitting o underfitting

**Bosques**

Muy probable que si underfitting, porque no logra capturar patrones útiles, da predicciones generales o erróneas.

\
**Árbol**

Overfitting posiblemente sí, aunque leve.

####MEJORAR

In [ ]:
def calcular_metricas (y_test, y_pred):
  exactitud = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred)

  metricas = {'Exactitud':round(exactitud, 2),
              'Precisión':round(precision, 2),
              'recall':round(recall, 2),
              'F1':round(f1, 2)
              }
  return metricas

#####Modelo bosques

In [ ]:
importances = modelo_bosques.feature_importances_

In [ ]:
feature_importances_ = pd.DataFrame({'Features':X.columns, 'Importances':(importances*100).round(2)}).sort_values('Importances', ascending = False)
feature_importances_

In [ ]:
len(datos_cod.columns)

In [ ]:
resultados = pd.DataFrame(index =['Exactitud', 'Precision', 'Recall', 'F1-score'])
model_features= RandomForestClassifier(max_depth = 5, random_state=21)
ct_features = [ i if i != 0 else 1 for i in range (0,30,2)]

for i in ct_features:
  select_features = feature_importances_['Features'].values[:i]
  X_train_sel = X_train[select_features]
  X_test_sel = X_test[select_features]
  model_features.fit(X_train_sel, y_train)
  y_pred = model_features.predict(X_test_sel)

  metricas = calcular_metricas(y_test, y_pred)
  resultados[i] = list(metricas.values())

resultados

In [ ]:
select_features = feature_importances_['Features'].values[:14]
X_selected_features = X[select_features]

X_selected_features.head(1)

In [ ]:
X_train_final = X_train[select_features]
X_test_final = X_test[select_features]

modelo_bosque_final= RandomForestClassifier(max_depth=5, random_state=21)
modelo_bosque_final.fit(X_train_final, y_train)

In [ ]:
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['sqrt', 'log2']
}

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [ ]:
cv = KFold(n_splits= 5, shuffle=True, random_state=21)
grid_rf = GridSearchCV(
    estimator=modelo_bosque_final,
    param_grid=param_grid,
    cv=cv,
    scoring='f1',
)

grid_rf.fit(X_train, y_train)

In [ ]:
print("Mejores parámetros para Random Forest:")
print(grid_rf.best_params_)


In [ ]:
modelo_bosques_final = grid_rf.best_estimator_

from sklearn.metrics import classification_report

y_pred_rf = modelo_bosques_final.predict(X_test)
print("Reporte del mejor modelo de Random Forest:")
print(classification_report(y_test, y_pred_rf, digits=2))

print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))

In [ ]:
##USAR EL MODELO
y_pred_final = modelo_bosque_final.predict(X_test_final)
metricas_finales = calcular_metricas(y_test, y_pred_final)

print(metricas_finales)


#####Modelo árbol

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
importancia_vars = modelo_arbol.feature_importances_
importancia_vars

In [ ]:
resultados = pd.DataFrame(index =['Exactitud', 'Precision', 'Recall', 'F1-score'])
model_features2= DecisionTreeClassifier(max_depth = 5, random_state=21)
ct_features = [ i if i != 0 else 1 for i in range (0,30,2)]

for i in ct_features:
  select_features = feature_importances_['Features'].values[:i]
  X_train_sel = X_train[select_features]
  X_test_sel = X_test[select_features]
  model_features2.fit(X_train_sel, y_train)
  y_pred2 = model_features2.predict(X_test_sel)

  metricas = calcular_metricas(y_test, y_pred2)
  resultados[i] = list(metricas.values())

resultados

In [ ]:
select_features2 = feature_importances_['Features'].values[:6]
X_selected_features2 = X[select_features2]

X_selected_features2.head(1)

In [ ]:
X_train_final2 = X_train2[select_features2]
X_test_final2 = X_test2[select_features2]

modelo_arbol_final= DecisionTreeClassifier(max_depth=5, random_state=21)
modelo_arbol_final.fit(X_train_final2, y_train2)

In [ ]:
param_grid = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5, 10],
    }

In [ ]:
cv = KFold(n_splits= 5, shuffle=True, random_state=21)
model_grid = GridSearchCV(modelo_arbol_final,
                          param_grid = param_grid, scoring = 'f1', cv = cv )

model_grid.fit(X_train2, y_train2)

In [ ]:
print("Mejores parámetros:", model_grid.best_params_)

In [ ]:
modelo_arbol_final = DecisionTreeClassifier(
    max_depth=5,
    min_samples_leaf=1,
    min_samples_split=2,
    random_state=21
)

modelo_arbol_final.fit(X_train2, y_train2)

In [ ]:
y_val_pred = modelo_arbol.predict(X_val)

In [ ]:
y_val_pred = modelo_arbol_final.predict(X_test2)

print("Reporte del mejor modelo de Árbol de decisión:")
print(classification_report(y_test2, y_val_pred, digits=2))

print("Matriz de confusión:")
print(confusion_matrix(y_test2, y_pred2))


###COMPARACIÓN 2

| Aspecto                        | Ganador           | Comentario                             |
| ------------------------------ | ----------------- | -------------------------------------- |
| Accuracy general               | Random Forest     | 80% vs 79%                             |
| Clase 0 (mayoría)              | Random Forest     | Clasifica mejor sin fallos             |
| Clase 1 (minoría)              | Árbol de decisión | Mejor recall y F1-score para positivos |
| Balance entre clases           | Árbol de decisión | Macro avg más alto                     |
| Menos falsos negativos clase 1 | Árbol de decisión | Aunque falla más en clase 0            |


#4.Interpretación y conclusiones

## Análisis de la importancia de las variables

Random Forest: Utiliza la importancia de las variables proporcionada por el modelo. Random Forest calcula la importancia basándose en cómo cada variable contribuye a la reducción de la impureza durante las divisiones de los árboles.

Otros Modelos: Dependiendo del modelo elegido, considera el análisis de métricas específicas para comprender la relevancia de las variables. Por ejemplo, coeficientes en modelos lineales, pesos en redes neuronales, o la importancia relativa en técnicas de boosting (como XGBoost).

### Random Forest

**Análisis**

---
Variables más importantes (top 14 usadas por el modelo):

| Variable                         | Importancia (%) | Correlación con cancelación |
| -------------------------------- | --------------- | --------------------------- |
| `contrato_Month-to-month`        | **21.41**       | **+0.396**                  |
| `meses_contrato`                 | **16.64**       | **−0.344**                  |
| `gastos_totales`                 | **12.53**       | **−0.194**                  |
| `proveedor_internet_Fiber optic` | **9.10**        | **+0.301**                  |
| `cuentas_diarias`                | 6.98            | +0.190                      |
| `gastos_mensuales`               | 6.32            | +0.190                      |
| `contrato_Two year`              | 5.91            | **−0.295**                  |
| `forma_pago_Electronic check`    | 5.33            | **+0.295**                  |
| `proveedor_internet_No`          | 3.01            | **−0.224**                  |
| `contrato_One year`              | 2.22            | **−0.174**                  |
| `soporte_tecnico`                | 1.99            | −0.160                      |
| `proveedor_internet_DSL`         | 1.96            | **−0.120**                  |
| `facturacion_electronica`        | 1.37            | +0.187                      |
| `seguridad_linea`                | 1.24            | −0.167                      |





**Factores que aumentan la probabilidad de cancelación:**
- Contrato mes a mes (contrato_Month-to-month)
- Pago por cheque electrónico (forma_pago_Electronic check)
- Internet por fibra óptica
- Gastos elevados y cuentas diarias altas


**Factores que disminuyen la cancelación:**
- Contrato de mayor duración (One year, Two year)
- Más meses con la empresa
- Soporte técnico
- Seguridad en la línea
- No tener proveedor de internet


Clientes con contratos flexibles, pagos electrónicos y uso intenso tienen mayor riesgo de cancelar.

### Modelo árbol

**Análisis**

---
Variables más importantes (top 6 usadas por el modelo):

| Variable                         | Importancia (%) | Correlación con cancelación |
| -------------------------------- | --------------- | --------------------------- |
| `contrato_Month-to-month`        | **21.41**       | **+0.396**                  |
| `meses_contrato`                 | **16.64**       | **−0.344**                  |
| `gastos_totales`                 | **12.53**       | **−0.194**                  |
| `proveedor_internet_Fiber optic` | **9.10**        | **+0.301**                  |
| `cuentas_diarias`                | **6.98**        | **+0.190**                  |
| `gastos_mensuales`               | **6.32**        | **+0.190**                     |

Coincide en gran medida con Random Forest, lo que refuerza la validez de estos factores.

**Conclusión de este análisis**

---
Variables clave en la predicción de cancelación:
- **Contrato mes a mes:** el predictor más fuerte de cancelación. Clientes sin compromiso de permanencia son más propensos a irse.

- **Duración con la empresa y tipo de contrato:** entre más tiempo y mayor compromiso, menor probabilidad de cancelar.

- **Gasto y uso del servicio:** quienes gastan más o tienen más actividad tienden a cancelar, posiblemente por percepciones de costo-beneficio.

- **Método de pago:** pagos automáticos están asociados con menor cancelación; electrónicos (manuales) con mayor cancelación.

- **Tipo de internet:** la fibra óptica, aunque más rápida, se asocia con mayor cancelación — podría estar relacionada con expectativas no cumplidas o competencia.


##Conclusión

**Introducción**
---
El análisis tuvo como propósito identificar los factores que más influyen en la cancelación de clientes, mediante modelos de clasificación y análisis de variables. Se entrenaron modelos de Árbol de Decisión y Random Forest, evaluando su rendimiento y extrayendo la importancia de cada variable para interpretar los resultados.

**Desarrollo**
---

**Rendimiento de los modelos**


---



| Modelo            | Exactitud | Precisión | Recall | F1-score |
| ----------------- | --------- | --------- | ------ | -------- |
| Árbol de Decisión | 0.74      | 0.49      | 0.48   | 0.49     |
| Random Forest     | 0.66      | 0.51      | 0.42   | 0.46     |

El Árbol de Decisión tuvo mejor rendimiento general, destacando por su capacidad de interpretar con claridad los factores más relevantes. Sin embargo, el modelo de Random Forest ofreció mayor robustez y una mejor comprensión del conjunto completo de variables.

\

**Factores que más influyen en la cancelación**

---

Ambos modelos coinciden en señalar como factores clave los siguientes:

*Variables asociadas con mayor riesgo de cancelación:*
- Contrato mensual (contrato_Month-to-month)
  - Clientes sin compromiso de permanencia son significativamente más propensos a cancelar.

- Forma de pago: Cheque electrónico (forma_pago_Electronic check)
 - Asociado con mayor cancelación; posiblemente clientes menos bancarizados o más sensibles a procesos manuales.

- Proveedor de internet por fibra óptica (proveedor_internet_Fiber optic)
  - Puede estar relacionado con expectativas no satisfechas o mayor competencia en este tipo de servicio.

- Altos gastos y uso diario (gastos_mensuales, cuentas_diarias)
  - Clientes con mayor uso pueden estar más conscientes del costo y evaluar alternativas.

*Variables que reducen la probabilidad de cancelación:*
- Contratos a largo plazo (contrato_Two year, contrato_One year)
  - Generan compromiso con la empresa y reducen la fuga.

- Mayor tiempo con la empresa (meses_contrato)
 - Clientes con mayor antigüedad tienen menor tasa de cancelación.

- Soporte técnico y servicios adicionales (soporte_tecnico, seguridad_linea)
  - Mejoran la percepción del valor del servicio.



**Estrategias de retención sugeridas**


---


Basado en los factores encontrados, se proponen las siguientes estrategias para disminuir la cancelación:

- Incentivar contratos a largo plazo
  - Ofrecer beneficios (descuentos, servicios extra, acceso prioritario) por migrar a contratos anuales o bianuales.

- Fomentar métodos de pago automáticos
  - Promocionar pagos con tarjeta o transferencia automática como más seguros y cómodos.

- Ofrecer bonificaciones por cambiar a estos métodos.
  - Fortalecer soporte técnico y servicios adicionales

- Mejorar la atención al cliente y soporte para usuarios con problemas técnicos.
  - Promover servicios de seguridad y respaldo como valor agregado.

- Monitorear a clientes de alto uso
 - Identificar clientes con uso intensivo y gastos elevados, para brindar atención personalizada, ofertas, o asesoramiento antes de que decidan cancelar.

- Seguimiento proactivo a usuarios con contrato mensual
  - Campañas de fidelización o encuestas de satisfacción orientadas a usuarios sin compromiso contractual.

**Conclusión**
---
Este proyecto no solo nos permitió aplicar técnicas de machine learning, sino que también nos enseñó que los datos cuentan historias. En este caso, la historia es clara: el compromiso del cliente, la forma en que paga y cómo usa el servicio influyen fuertemente en su decisión de cancelar o quedarse.

Entender estos factores puede ayudar a cualquier empresa de servicios a tomar mejores decisiones, mejorar su relación con los clientes y, por supuesto, reducir las cancelaciones.